In [23]:
import pandas as pd
import numpy as np
from itertools import product

class QuizGridSearchAnalyzer:
    def __init__(self):
        # 비용/수익 설정
        self.costs = {
            'ad_revenue': 1,           # 광고 수익
            'success_reward': 3,       # 일반 보상
            'retry_success_reward': 1, # 재도전 성공 보상
            'streak_reward': 100,      # 7일 연속 참여 보상
            'extra_reward': 3          # 추가 보상
        }
        
        # 7일 연속 참여 분포 (17.5% 트리거 확률)
        self.streak_7_trigger_rate = 0.175
        self.streak_7_distribution = {
            1: 0.4911, 2: 0.1823, 3: 0.0976, 4: 0.0589, 5: 0.0421,
            6: 0.0283, 7: 0.0217, 8: 0.0148, 9: 0.0154, 10: 0.0123,
            11: 0.0161, 12: 0.0193
        }
        
        self.daily_users = 100000
    
    def calculate_success_scenario(self, p_correct, p_watch_ad, p_extra_reward, p_retry):
        """정답 즉시 성공 시나리오 계산"""
        # 광고 시청 수 (정답자 중 광고 시청하는 비율)
        ad_views = p_watch_ad
        
        # 추가 보상 획득 수 (광고 시청자 중 추가 보상 받는 비율)
        extra_rewards = p_watch_ad * p_extra_reward
        
        # 수익/비용
        revenue = ad_views * self.costs['ad_revenue']
        cost = self.costs['success_reward'] + extra_rewards * self.costs['extra_reward']
        
        return {
            'ad_views': ad_views,
            'extra_rewards': extra_rewards,
            'revenue': revenue,
            'cost': cost,
            'profit': revenue - cost
        }
    
    def calculate_failure_scenario(self, p_correct, p_watch_ad, p_extra_reward, p_retry):
        """오답 후 재도전 시나리오 계산"""
        total_ad_views = 1  # 첫 강제 광고
        total_extra_rewards = 0
        total_retry_success = 0
        
        retry_chance = p_retry
        max_retries = 10
        
        for attempt in range(1, max_retries + 1):
            # 이번 시도에서 성공할 확률
            success_prob = retry_chance * p_correct
            
            if success_prob > 0.001:  # 무시할 수 있는 수준까지
                # 재도전 성공 시
                total_retry_success += success_prob
                
                # 성공 후 광고 시청 및 추가 보상
                ad_after_success = success_prob * p_watch_ad
                extra_after_success = ad_after_success * p_extra_reward
                
                total_ad_views += ad_after_success
                total_extra_rewards += extra_after_success
            
            # 실패하고 재도전할 확률
            fail_and_retry_prob = retry_chance * (1 - p_correct) * p_retry
            
            if fail_and_retry_prob < 0.001:  # 무시할 수 있는 수준
                break
            
            # 실패 시 추가 광고
            total_ad_views += fail_and_retry_prob
            
            # 다음 재도전 확률 업데이트
            retry_chance = fail_and_retry_prob
        
        # 수익/비용 계산
        revenue = total_ad_views * self.costs['ad_revenue']
        cost = total_retry_success * self.costs['retry_success_reward'] + \
               total_extra_rewards * self.costs['extra_reward']
        
        return {
            'ad_views': total_ad_views,
            'retry_success': total_retry_success,
            'extra_rewards': total_extra_rewards,
            'revenue': revenue,
            'cost': cost,
            'profit': revenue - cost
        }
    
    def calculate_7day_streak_metrics(self):
        """7일 연속 참여 보상 계산"""
        adjusted_users = self.daily_users * self.streak_7_trigger_rate
        
        total_streak_users = 0
        total_streak_cost = 0
        
        for streak_count, probability in self.streak_7_distribution.items():
            expected_users = adjusted_users * probability
            streak_cost = expected_users * streak_count * self.costs['streak_reward']
            
            total_streak_users += expected_users
            total_streak_cost += streak_cost
        
        return {
            'daily_streak_users': total_streak_users,
            'daily_streak_cost': total_streak_cost
        }
    
    def calculate_scenario_metrics(self, p_correct, p_watch_ad, p_extra_reward, p_retry):
        """단일 시나리오의 모든 지표 계산"""
        
        # 성공/실패 시나리오별 계산
        success_result = self.calculate_success_scenario(p_correct, p_watch_ad, p_extra_reward, p_retry)
        failure_result = self.calculate_failure_scenario(p_correct, p_watch_ad, p_extra_reward, p_retry)
        
        # 일일 사용자 분포
        daily_success_users = self.daily_users * p_correct
        daily_failure_users = self.daily_users * (1 - p_correct)
        
        # 일일 집계 지표
        일일_광고시청수 = int(daily_success_users * success_result['ad_views'] + 
                         daily_failure_users * failure_result['ad_views'])
        
        일일_참여수 = self.daily_users
        
        일일_일반보상지급수 = int(daily_success_users)
        
        일일_재도전성공수 = int(daily_failure_users * failure_result['retry_success'])
        
        일일_추가보상수 = int(daily_success_users * success_result['extra_rewards'] +
                         daily_failure_users * failure_result['extra_rewards'])
        
        # 7일 연속 참여 보상
        streak_metrics = self.calculate_7day_streak_metrics()
        일일_7일연속보상수 = int(streak_metrics['daily_streak_users'])
        
        # 보상액 계산
        일반보상액 = 일일_일반보상지급수 * self.costs['success_reward']
        재도전보상액 = 일일_재도전성공수 * self.costs['retry_success_reward']
        추가보상액 = 일일_추가보상수 * self.costs['extra_reward']
        연속보상액 = streak_metrics['daily_streak_cost']
        총보상액 = 일반보상액 + 재도전보상액 + 추가보상액 + 연속보상액
        
        # 매출액
        광고매출액 = 일일_광고시청수 * self.costs['ad_revenue']
        총매출액 = 광고매출액
        
        # 손익 계산
        일일순손익액 = 총매출액 - 총보상액
        주간게임손익액 = (광고매출액 - (일반보상액 + 재도전보상액 + 추가보상액)) * 7
        주간순손익액 = 주간게임손익액 - 연속보상액
        수익성여부 = '수익' if 주간순손익액 > 0 else '손실'
        
        return {
            '정답률': f"{p_correct:.1%}",
            '광고시청률': f"{p_watch_ad:.1%}",
            '추가보상률': f"{p_extra_reward:.1%}",
            '재도전률': f"{p_retry:.1%}",
            '일일_광고시청수': 일일_광고시청수,
            '일일_참여수': 일일_참여수,
            '일일_일반보상지급수': 일일_일반보상지급수,
            '일일_재도전성공수': 일일_재도전성공수,
            '일일_추가보상수': 일일_추가보상수,
            '일일_7일연속보상수': 일일_7일연속보상수,
            '일반보상액': 일반보상액,
            '재도전보상액': 재도전보상액,
            '추가보상액': 추가보상액,
            '7일연속보상액': int(연속보상액),
            '총보상액': int(총보상액),
            '광고매출액': 광고매출액,
            '총매출액': 총매출액,
            '일일순손익액': int(일일순손익액),
            '주간순손익액': int(주간순손익액),
            '수익성여부': 수익성여부
        }
    
    def run_grid_search(self):
        """전체 Grid Search 실행"""
        
        # Grid 범위 정의
        correct_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        watch_ad_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        extra_reward_rates = [0.5, 0.6, 0.7, 0.8, 0.9]
        retry_rates = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        
        print(f"Grid Search 시작: {len(correct_rates)}×{len(watch_ad_rates)}×{len(extra_reward_rates)}×{len(retry_rates)} = {len(correct_rates)*len(watch_ad_rates)*len(extra_reward_rates)*len(retry_rates)}개 시나리오")
        
        results = []
        scenario_id = 1
        
        # 모든 조합에 대해 계산
        for p_correct, p_watch_ad, p_extra_reward, p_retry in product(
            correct_rates, watch_ad_rates, extra_reward_rates, retry_rates
        ):
            metrics = self.calculate_scenario_metrics(p_correct, p_watch_ad, p_extra_reward, p_retry)
            
            # 시나리오 ID 추가
            metrics['시나리오_ID'] = f"S{scenario_id:04d}"
            
            results.append(metrics)
            scenario_id += 1
            
            # 진행상황 출력
            if scenario_id % 200 == 0:
                print(f"진행상황: {scenario_id-1}개 완료...")
        
        # DataFrame 생성 (컬럼 순서 조정)
        column_order = [
            '시나리오_ID', '정답률', '광고시청률', '추가보상률', '재도전률',
            '일일_광고시청수', '일일_참여수', '일일_일반보상지급수', '일일_재도전성공수', 
            '일일_추가보상수', '일일_7일연속보상수',
            '일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액',
            '광고매출액', '총매출액', '일일순손익액', '주간순손익액', '수익성여부'
        ]
        
        df = pd.DataFrame(results)[column_order]
        
        print(f"Grid Search 완료: {len(df)}개 시나리오 생성")
        print(f"수익 시나리오: {len(df[df['수익성여부'] == '수익'])}개")
        print(f"손실 시나리오: {len(df[df['수익성여부'] == '손실'])}개")
        
        return df

def main():
    """메인 실행 함수"""
    
    # 분석기 생성 및 실행
    analyzer = QuizGridSearchAnalyzer()
    df = analyzer.run_grid_search()
    
    # 기본 통계 출력
    print("\n=== 기본 통계 ===")
    print(f"전체 시나리오 수: {len(df):,}")
    
    # 컬럼 존재 여부 확인
    print(f"DataFrame 컬럼: {list(df.columns)}")
    
    if '수익성여부' in df.columns:
        print(f"수익 시나리오 수: {len(df[df['수익성여부'] == '수익']):,}")
        print(f"수익률: {len(df[df['수익성여부'] == '수익'])/len(df)*100:.1f}%")
    
    if '주간순손익액' in df.columns:
        print(f"\n주간 순손익 범위:")
        print(f"최대 수익: {df['주간순손익액'].max():,}원")
        print(f"최대 손실: {df['주간순손익액'].min():,}원")
        print(f"평균: {df['주간순손익액'].mean():,.0f}원")
    
    # 컬럼 확인 후 상위 시나리오 출력
    required_columns = ['시나리오_ID', '광고매출', '7일연속보상', '추가보상', 
                       '정답률', '광고시청률', '추가보상률', '재도전률', '주간순손익액']
    
    if all(col in df.columns for col in required_columns):
        # 상위 10개 수익 시나리오
        print("\n=== 상위 10개 수익 시나리오 ===")
        top10 = df.nlargest(10, '주간순손익액')[required_columns]
        print(top10.to_string(index=False))
    else:
        missing_cols = [col for col in required_columns if col not in df.columns]
        print(f"❌ 필요한 컬럼이 없습니다: {missing_cols}")
        print("샘플 데이터 (처음 5행):")
        print(df.head().to_string())
    
    # 기본 시나리오 찾기
    base_conditions = [
        ('광고매출', 1), ('7일연속보상', 100), ('추가보상', 3),
        ('정답률', '50.0%'), ('광고시청률', '50.0%'), 
        ('추가보상률', '70.0%'), ('재도전률', '50.0%')
    ]
    
    base_scenario = df.copy()
    all_conditions_exist = True
    
    for col, val in base_conditions:
        if col in df.columns:
            base_scenario = base_scenario[base_scenario[col] == val]
        else:
            print(f"❌ 컬럼 '{col}'이 존재하지 않습니다.")
            all_conditions_exist = False
            break
    
    if all_conditions_exist and not base_scenario.empty:
        print(f"\n=== 기본 시나리오 ({base_scenario.iloc[0]['시나리오_ID']}) ===")
        print(f"설정: 광고매출={base_scenario.iloc[0]['광고매출']}원, 7일연속보상={base_scenario.iloc[0]['7일연속보상']}원, 추가보상={base_scenario.iloc[0]['추가보상']}원")
        print(f"확률: {base_scenario.iloc[0]['정답률']}, {base_scenario.iloc[0]['광고시청률']}, {base_scenario.iloc[0]['추가보상률']}, {base_scenario.iloc[0]['재도전률']}")
        if '주간순손익액' in df.columns:
            print(f"주간 순손익: {base_scenario.iloc[0]['주간순손익액']:,}원")
        if '수익성여부' in df.columns:
            print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 비용/매출 변수별 수익 시나리오 분포
    if '수익성여부' in df.columns:
        profitable_df = df[df['수익성여부'] == '수익']
        
        if len(profitable_df) > 0:
            print(f"\n=== 비용/매출 변수별 수익 시나리오 분포 ===")
            
            for var in ['광고매출', '7일연속보상', '추가보상']:
                if var in df.columns:
                    var_dist = profitable_df[var].value_counts().sort_index()
                    print(f"{var}별 수익 시나리오 수:")
                    for value, count in var_dist.items():
                        print(f"  {value}: {count:,}개")
                    print()
    
    # CSV 저장
    filename = 'quiz_grid_search_results_extended.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n결과가 '{filename}'에 저장되었습니다.")
    
    return df원: {count:,}개")
        
        print(f"\n7일연속보상별 수익 시나리오 수:")
        streak_reward_dist = profitable_df['7일연속보상'].value_counts().sort_index()
        for reward, count in streak_reward_dist.items():
            print(f"  {reward}원: {count:,}개")
        
        print(f"\n추가보상별 수익 시나리오 수:")
        extra_reward_dist = profitable_df['추가보상'].value_counts().sort_index()
        for reward, count in extra_reward_dist.items():
            print(f"  {reward}원: {count:,}개")
    
    # CSV 저장
    df.to_csv('quiz_grid_search_results_extended.csv', index=False, encoding='utf-8-sig')
    print(f"\n결과가 'quiz_grid_search_results_extended.csv'에 저장되었습니다.")
    
    return df

def extract_best_scenarios(df, base_scenario_params=None, n=10, sort_by='주간순손익액', improvement_threshold=0):
    """
    기존 대비 가장 좋은 시나리오를 n개 추출하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    base_scenario_params : dict, optional
        기준 시나리오 파라미터. None이면 기본값 사용
        예: {'광고매출': 1, '7일연속보상': 100, '추가보상': 3, 
             '정답률': '50.0%', '광고시청률': '50.0%', '추가보상률': '70.0%', '재도전률': '50.0%'}
    n : int
        추출할 시나리오 개수 (기본값: 10)
    sort_by : str
        정렬 기준 컬럼 (기본값: '주간순손익액')
    improvement_threshold : int
        최소 개선 금액 기준 (기본값: 0원)
    
    Returns:
    --------
    dict : 분석 결과를 포함한 딕셔너리
    """
    
    # 기본 시나리오 파라미터 설정
    if base_scenario_params is None:
        base_scenario_params = {
            '광고매출': 1,
            '7일연속보상': 100,
            '추가보상': 3,
            '정답률': '50.0%',
            '광고시청률': '50.0%',
            '추가보상률': '70.0%',
            '재도전률': '50.0%'
        }
    
    # 기준 시나리오 찾기
    base_scenario = df.copy()
    for key, value in base_scenario_params.items():
        base_scenario = base_scenario[base_scenario[key] == value]
    
    if base_scenario.empty:
        print("❌ 기준 시나리오를 찾을 수 없습니다.")
        print(f"검색 조건: {base_scenario_params}")
        return None
    
    base_profit = base_scenario.iloc[0][sort_by]
    base_scenario_id = base_scenario.iloc[0]['시나리오_ID']
    
    print(f"=== 기준 시나리오 ({base_scenario_id}) ===")
    print(f"설정: 광고매출={base_scenario.iloc[0]['광고매출']}원, 7일연속보상={base_scenario.iloc[0]['7일연속보상']}원, 추가보상={base_scenario.iloc[0]['추가보상']}원")
    print(f"확률: {base_scenario.iloc[0]['정답률']}, {base_scenario.iloc[0]['광고시청률']}, {base_scenario.iloc[0]['추가보상률']}, {base_scenario.iloc[0]['재도전률']}")
    print(f"기준 {sort_by}: {base_profit:,}원")
    print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 개선된 시나리오 필터링
    improved_scenarios = df[df[sort_by] > base_profit + improvement_threshold].copy()
    
    if improved_scenarios.empty:
        print(f"\n❌ 개선 기준({improvement_threshold:,}원)을 만족하는 시나리오가 없습니다.")
        return {
            'base_scenario': base_scenario.iloc[0],
            'improved_scenarios': pd.DataFrame(),
            'analysis': {}
        }
    
    # 개선 금액 계산
    improved_scenarios['개선금액'] = improved_scenarios[sort_by] - base_profit
    improved_scenarios['개선배수'] = improved_scenarios[sort_by] / base_profit if base_profit != 0 else float('inf')
    
    # 상위 n개 추출
    best_scenarios = improved_scenarios.nlargest(n, sort_by).copy()
    
    print(f"\n=== 상위 {len(best_scenarios)}개 개선 시나리오 ===")
    print(f"총 {len(improved_scenarios):,}개 시나리오가 기준보다 우수함")
    
    # 결과 테이블 출력
    display_columns = [
        '시나리오_ID', '광고매출', '7일연속보상', '추가보상',
        '정답률', '광고시청률', '추가보상률', '재도전률', 
        sort_by, '개선금액', '수익성여부'
    ]
    
    print(best_scenarios[display_columns].to_string(index=False))
    
    # 변수별 개선 영향 분석
    print(f"\n=== 변수별 개선 영향 분석 ===")
    
    # 각 변수의 기준값 대비 변화량 계산
    variable_analysis = {}
    
    # 비용/매출 변수 분석
    for var in ['광고매출', '7일연속보상', '추가보상']:
        base_value = base_scenario.iloc[0][var]
        var_changes = best_scenarios[var] - base_value
        variable_analysis[var] = {
            '기준값': base_value,
            '평균변화': var_changes.mean(),
            '최대값': best_scenarios[var].max(),
            '최소값': best_scenarios[var].min(),
            '변화범위': f"{var_changes.min():+} ~ {var_changes.max():+}"
        }
        print(f"{var}: 기준 {base_value} → 범위 {best_scenarios[var].min()}~{best_scenarios[var].max()} (평균변화: {var_changes.mean():+.1f})")
    
    # 확률 변수 분석
    print(f"\n확률 변수 분포:")
    for var in ['정답률', '광고시청률', '추가보상률', '재도전률']:
        var_dist = best_scenarios[var].value_counts().sort_index()
        most_common = var_dist.index[0]
        print(f"{var}: 최빈값 {most_common} ({var_dist.iloc[0]}회), 전체 {len(var_dist)}개 값")
    
    # 달성 가능성 분석
    print(f"\n=== 달성 가능성 분석 ===")
    
    # 기준 대비 변화량 기준으로 난이도 분류
    feasibility_analysis = []
    
    for idx, row in best_scenarios.iterrows():
        changes = 0
        difficulty_score = 0
        
        # 비용/매출 변수 변화 체크
        for var in ['광고매출', '7일연속보상', '추가보상']:
            base_val = base_scenario.iloc[0][var]
            current_val = row[var]
            if current_val != base_val:
                changes += 1
                # 변화량에 따른 난이도 점수
                if var == '광고매출':
                    difficulty_score += abs(current_val - base_val) * 1  # 광고 수익 변경 난이도
                elif var == '7일연속보상':
                    difficulty_score += abs(current_val - base_val) / 10 * 2  # 보상 정책 변경 난이도
                elif var == '추가보상':
                    difficulty_score += abs(current_val - base_val) * 1.5  # 추가 보상 변경 난이도
        
        # 확률 변수 변화 체크 (백분율을 숫자로 변환)
        prob_vars = ['정답률', '광고시청률', '추가보상률', '재도전률']
        base_probs = [float(base_scenario.iloc[0][var].rstrip('%')) for var in prob_vars]
        current_probs = [float(row[var].rstrip('%')) for var in prob_vars]
        
        for i, var in enumerate(prob_vars):
            if current_probs[i] != base_probs[i]:
                changes += 1
                # 확률 변경 난이도 (변화량에 비례)
                difficulty_score += abs(current_probs[i] - base_probs[i]) / 10
        
        # 난이도 분류
        if difficulty_score <= 2:
            difficulty = "쉬움"
        elif difficulty_score <= 5:
            difficulty = "보통"
        else:
            difficulty = "어려움"
        
        feasibility_analysis.append({
            '시나리오_ID': row['시나리오_ID'],
            '변경_변수_수': changes,
            '난이도_점수': round(difficulty_score, 1),
            '달성_난이도': difficulty,
            '개선금액': row['개선금액']
        })
    
    feasibility_df = pd.DataFrame(feasibility_analysis)
    
    # 난이도별 분포
    difficulty_dist = feasibility_df['달성_난이도'].value_counts()
    print(f"달성 난이도 분포:")
    for difficulty, count in difficulty_dist.items():
        avg_improvement = feasibility_df[feasibility_df['달성_난이도'] == difficulty]['개선금액'].mean()
        print(f"  {difficulty}: {count}개 (평균 개선금액: {avg_improvement:,.0f}원)")
    
    # 가장 달성하기 쉬운 상위 3개 추천
    easy_scenarios = feasibility_df[feasibility_df['달성_난이도'] == '쉬움'].nlargest(3, '개선금액')
    if not easy_scenarios.empty:
        print(f"\n💡 달성하기 쉬운 추천 시나리오 (상위 3개):")
        for idx, scenario in easy_scenarios.iterrows():
            scenario_detail = best_scenarios[best_scenarios['시나리오_ID'] == scenario['시나리오_ID']].iloc[0]
            print(f"  {scenario['시나리오_ID']}: {scenario['개선금액']:,}원 개선 (변경 {scenario['변경_변수_수']}개 변수)")
            print(f"    설정: 광고={scenario_detail['광고매출']}, 연속보상={scenario_detail['7일연속보상']}, 추가보상={scenario_detail['추가보상']}")
            print(f"    확률: {scenario_detail['정답률']}, {scenario_detail['광고시청률']}, {scenario_detail['추가보상률']}, {scenario_detail['재도전률']}")
    
    return {
        'base_scenario': base_scenario.iloc[0],
        'best_scenarios': best_scenarios,
        'improved_count': len(improved_scenarios),
        'variable_analysis': variable_analysis,
        'feasibility_analysis': feasibility_df,
        'easy_recommendations': easy_scenarios if not easy_scenarios.empty else pd.DataFrame()
    }

def analyze_scenario_comparison(df, scenario_ids):
    """
    특정 시나리오들을 상세 비교 분석하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    scenario_ids : list
        비교할 시나리오 ID 리스트
    """
    scenarios = df[df['시나리오_ID'].isin(scenario_ids)].copy()
    
    if scenarios.empty:
        print("❌ 해당 시나리오들을 찾을 수 없습니다.")
        return
    
    print(f"=== 시나리오 상세 비교 ({len(scenarios)}개) ===")
    
    # 주요 지표 비교
    comparison_columns = [
        '시나리오_ID', '광고매출', '7일연속보상', '추가보상',
        '정답률', '광고시청률', '추가보상률', '재도전률',
        '일일_광고시청수', '일일순손익액', '주간순손익액', '수익성여부'
    ]
    
    print(scenarios[comparison_columns].to_string(index=False))
    
    # 수익 구조 분석
    print(f"\n=== 수익 구조 분석 ===")
    revenue_columns = ['광고매출액', '총매출액']
    cost_columns = ['일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액']
    
    for idx, row in scenarios.iterrows():
        print(f"\n{row['시나리오_ID']}:")
        print(f"  매출: 광고 {row['광고매출액']:,}원")
        print(f"  비용: 일반 {row['일반보상액']:,}원 + 재도전 {row['재도전보상액']:,}원 + 추가 {row['추가보상액']:,}원 + 연속 {row['7일연속보상액']:,}원 = {row['총보상액']:,}원")
        print(f"  순이익: {row['주간순손익액']:,}원")

# 실행
if __name__ == "__main__":
    df = main()
    
    # DataFrame이 성공적으로 생성되었는지 확인
    if df is not None and len(df) > 0:
        print("\n" + "="*80)
        
        # 예시: 상위 10개 시나리오 추출
        if 'extract_best_scenarios' in globals():
            try:
                best_analysis = extract_best_scenarios(df, n=10)
                
                if best_analysis:
                    print("\n✅ 기본 시나리오 분석 완료")
                else:
                    print("\n❌ 기본 시나리오 분석 실패")
            except Exception as e:
                print(f"\n❌ 기본 시나리오 분석 중 오류: {e}")
        
        # 예시: 특정 조건으로 시나리오 추출 (컬럼 존재 확인)
        print("\n" + "="*80)
        
        # 커스텀 기준 시나리오가 있는지 확인
        required_cols = ['광고매출', '7일연속보상', '추가보상', '정답률', '광고시청률', '추가보상률', '재도전률']
        if all(col in df.columns for col in required_cols):
            try:
                custom_base = {
                    '광고매출': 2,
                    '7일연속보상': 50,
                    '추가보상': 2,
                    '정답률': '60.0%',
                    '광고시청률': '60.0%',
                    '추가보상률': '80.0%',
                    '재도전률': '40.0%'
                }
                custom_analysis = extract_best_scenarios(df, base_scenario_params=custom_base, n=5, improvement_threshold=100000)
                
                if custom_analysis:
                    print("\n✅ 커스텀 시나리오 분석 완료")
                else:
                    print("\n❌ 커스텀 시나리오 분석 실패")
            except Exception as e:
                print(f"\n❌ 커스텀 시나리오 분석 중 오류: {e}")
        else:
            missing_cols = [col for col in required_cols if col not in df.columns]
            print(f"\n⚠️ 커스텀 분석을 위한 필수 컬럼 누락: {missing_cols}")
    else:
        print("\n❌ DataFrame 생성 실패")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 368)

In [24]:
df.to_excel('./quiz.xlsx')

### 시나리오 분석

In [25]:
def main():
    """메인 실행 함수"""
    
    # 분석기 생성 및 실행
    analyzer = QuizGridSearchAnalyzer()
    
    # 먼저 작은 샘플로 테스트
    print("=== 샘플 테스트 ===")
    test_metrics = analyzer.calculate_scenario_metrics(0.5, 0.5, 0.7, 0.5, 1, 100, 3)
    print(f"테스트 결과 키: {list(test_metrics.keys())}")
    print(f"광고매출 포함 여부: {'광고매출' in test_metrics}")
    print(f"7일연속보상 포함 여부: {'7일연속보상' in test_metrics}")
    print(f"추가보상 포함 여부: {'추가보상' in test_metrics}")
    
    # 전체 Grid Search 실행
    df = analyzer.run_grid_search()
    
    # 기본 통계 출력
    print("\n=== 기본 통계 ===")
    print(f"전체 시나리오 수: {len(df):,}")
    
    # 컬럼 존재 여부 확인
    print(f"DataFrame 컬럼: {list(df.columns)}")
    
    # 비용/매출 변수 확인
    cost_vars = ['광고매출', '7일연속보상', '추가보상']
    missing_cost_vars = [var for var in cost_vars if var not in df.columns]
    
    if missing_cost_vars:
        print(f"❌ 누락된 비용/매출 변수: {missing_cost_vars}")
    else:
        print(f"✅ 모든 비용/매출 변수 존재")
    
    if '수익성여부' in df.columns:
        print(f"수익 시나리오 수: {len(df[df['수익성여부'] == '수익']):,}")
        print(f"수익률: {len(df[df['수익성여부'] == '수익'])/len(df)*100:.1f}%")
    
    if '주간순손익액' in df.columns:
        print(f"\n주간 순손익 범위:")
        print(f"최대 수익: {df['주간순손익액'].max():,}원")
        print(f"최대 손실: {df['주간순손익액'].min():,}원")
        print(f"평균: {df['주간순손익액'].mean():,.0f}원")
    
    # 컬럼 확인 후 상위 시나리오 출력
    required_columns = ['시나리오_ID', '정답률', '광고시청률', '추가보상률', '재도전률', '주간순손익액']
    
    # 비용/매출 변수가 있으면 추가
    available_cost_vars = [var for var in cost_vars if var in df.columns]
    display_columns = ['시나리오_ID'] + available_cost_vars + ['정답률', '광고시청률', '추가보상률', '재도전률', '주간순손익액']
    
    # 실제 존재하는 컬럼만 필터링
    final_display_columns = [col for col in display_columns if col in df.columns]
    
    if '주간순손익액' in df.columns:
        # 상위 10개 수익 시나리오
        print(f"\n=== 상위 10개 수익 시나리오 ===")
        print(f"표시 컬럼: {final_display_columns}")
        top10 = df.nlargest(10, '주간순손익액')[final_display_columns]
        print(top10.to_string(index=False))
    
    # 기본 시나리오 찾기 (존재하는 컬럼만 사용)
    base_conditions = []
    
    if '광고매출' in df.columns:
        base_conditions.append(('광고매출', 1))
    if '7일연속보상' in df.columns:
        base_conditions.append(('7일연속보상', 100))
    if '추가보상' in df.columns:
        base_conditions.append(('추가보상', 3))
    if '정답률' in df.columns:
        base_conditions.append(('정답률', '50.0%'))
    if '광고시청률' in df.columns:
        base_conditions.append(('광고시청률', '50.0%'))
    if '추가보상률' in df.columns:
        base_conditions.append(('추가보상률', '70.0%'))
    if '재도전률' in df.columns:
        base_conditions.append(('재도전률', '50.0%'))
    
    if base_conditions:
        base_scenario = df.copy()
        for col, val in base_conditions:
            base_scenario = base_scenario[base_scenario[col] == val]
        
        if not base_scenario.empty:
            print(f"\n=== 기본 시나리오 ({base_scenario.iloc[0]['시나리오_ID']}) ===")
            
            # 설정 정보 출력 (존재하는 것만)
            cost_info = []
            if '광고매출' in df.columns:
                cost_info.append(f"광고매출={base_scenario.iloc[0]['광고매출']}원")
            if '7일연속보상' in df.columns:
                cost_info.append(f"7일연속보상={base_scenario.iloc[0]['7일연속보상']}원")
            if '추가보상' in df.columns:
                cost_info.append(f"추가보상={base_scenario.iloc[0]['추가보상']}원")
            
            if cost_info:
                print(f"설정: {', '.join(cost_info)}")
            
            # 확률 정보 출력
            prob_info = []
            for var in ['정답률', '광고시청률', '추가보상률', '재도전률']:
                if var in df.columns:
                    prob_info.append(str(base_scenario.iloc[0][var]))
            
            if prob_info:
                print(f"확률: {', '.join(prob_info)}")
            
            if '주간순손익액' in df.columns:
                print(f"주간 순손익: {base_scenario.iloc[0]['주간순손익액']:,}원")
            if '수익성여부' in df.columns:
                print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 비용/매출 변수별 수익 시나리오 분포
    if '수익성여부' in df.columns:
        profitable_df = df[df['수익성여부'] == '수익']
        
        if len(profitable_df) > 0:
            print(f"\n=== 비용/매출 변수별 수익 시나리오 분포 ===")
            
            for var in ['광고매출', '7일연속보상', '추가보상']:
                if var in df.columns:
                    var_dist = profitable_df[var].value_counts().sort_index()
                    print(f"{var}별 수익 시나리오 수:")
                    for value, count in var_dist.items():
                        print(f"  {value}: {count:,}개")
                    print()
    
    # CSV 저장
    filename = 'quiz_grid_search_results_extended.csv'
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n결과가 '{filename}'에 저장되었습니다.")
    
    return df


def extract_best_scenarios(df, base_scenario_params=None, n=10, sort_by='주간순손익액', improvement_threshold=0):
    """
    기존 대비 가장 좋은 시나리오를 n개 추출하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    base_scenario_params : dict, optional
        기준 시나리오 파라미터. None이면 기본값 사용
        예: {'광고매출': 1, '7일연속보상': 100, '추가보상': 3, 
             '정답률': '50.0%', '광고시청률': '50.0%', '추가보상률': '70.0%', '재도전률': '50.0%'}
    n : int
        추출할 시나리오 개수 (기본값: 10)
    sort_by : str
        정렬 기준 컬럼 (기본값: '주간순손익액')
    improvement_threshold : int
        최소 개선 금액 기준 (기본값: 0원)
    
    Returns:
    --------
    dict : 분석 결과를 포함한 딕셔너리
    """
    
    # 기본 시나리오 파라미터 설정
    if base_scenario_params is None:
        base_scenario_params = {
            '광고매출': 1,
            '7일연속보상': 100,
            '추가보상': 3,
            '정답률': '50.0%',
            '광고시청률': '50.0%',
            '추가보상률': '70.0%',
            '재도전률': '50.0%'
        }
    
    # 기준 시나리오 찾기
    base_scenario = df.copy()
    for key, value in base_scenario_params.items():
        base_scenario = base_scenario[base_scenario[key] == value]
    
    if base_scenario.empty:
        print("❌ 기준 시나리오를 찾을 수 없습니다.")
        print(f"검색 조건: {base_scenario_params}")
        return None
    
    base_profit = base_scenario.iloc[0][sort_by]
    base_scenario_id = base_scenario.iloc[0]['시나리오_ID']
    
    print(f"=== 기준 시나리오 ({base_scenario_id}) ===")
    print(f"설정: 광고매출={base_scenario.iloc[0]['광고매출']}원, 7일연속보상={base_scenario.iloc[0]['7일연속보상']}원, 추가보상={base_scenario.iloc[0]['추가보상']}원")
    print(f"확률: {base_scenario.iloc[0]['정답률']}, {base_scenario.iloc[0]['광고시청률']}, {base_scenario.iloc[0]['추가보상률']}, {base_scenario.iloc[0]['재도전률']}")
    print(f"기준 {sort_by}: {base_profit:,}원")
    print(f"수익성: {base_scenario.iloc[0]['수익성여부']}")
    
    # 개선된 시나리오 필터링
    improved_scenarios = df[df[sort_by] > base_profit + improvement_threshold].copy()
    
    if improved_scenarios.empty:
        print(f"\n❌ 개선 기준({improvement_threshold:,}원)을 만족하는 시나리오가 없습니다.")
        return {
            'base_scenario': base_scenario.iloc[0],
            'improved_scenarios': pd.DataFrame(),
            'analysis': {}
        }
    
    # 개선 금액 계산
    improved_scenarios['개선금액'] = improved_scenarios[sort_by] - base_profit
    improved_scenarios['개선배수'] = improved_scenarios[sort_by] / base_profit if base_profit != 0 else float('inf')
    
    # 상위 n개 추출
    best_scenarios = improved_scenarios.nlargest(n, sort_by).copy()
    
    print(f"\n=== 상위 {len(best_scenarios)}개 개선 시나리오 ===")
    print(f"총 {len(improved_scenarios):,}개 시나리오가 기준보다 우수함")
    
    # 결과 테이블 출력
    display_columns = [
        '시나리오_ID', '광고매출', '7일연속보상', '추가보상',
        '정답률', '광고시청률', '추가보상률', '재도전률', 
        sort_by, '개선금액', '수익성여부'
    ]
    
    print(best_scenarios[display_columns].to_string(index=False))
    
    # 변수별 개선 영향 분석
    print(f"\n=== 변수별 개선 영향 분석 ===")
    
    # 각 변수의 기준값 대비 변화량 계산
    variable_analysis = {}
    
    # 비용/매출 변수 분석
    for var in ['광고매출', '7일연속보상', '추가보상']:
        base_value = base_scenario.iloc[0][var]
        var_changes = best_scenarios[var] - base_value
        variable_analysis[var] = {
            '기준값': base_value,
            '평균변화': var_changes.mean(),
            '최대값': best_scenarios[var].max(),
            '최소값': best_scenarios[var].min(),
            '변화범위': f"{var_changes.min():+} ~ {var_changes.max():+}"
        }
        print(f"{var}: 기준 {base_value} → 범위 {best_scenarios[var].min()}~{best_scenarios[var].max()} (평균변화: {var_changes.mean():+.1f})")
    
    # 확률 변수 분석
    print(f"\n확률 변수 분포:")
    for var in ['정답률', '광고시청률', '추가보상률', '재도전률']:
        var_dist = best_scenarios[var].value_counts().sort_index()
        most_common = var_dist.index[0]
        print(f"{var}: 최빈값 {most_common} ({var_dist.iloc[0]}회), 전체 {len(var_dist)}개 값")
    
    # 달성 가능성 분석
    print(f"\n=== 달성 가능성 분석 ===")
    
    # 기준 대비 변화량 기준으로 난이도 분류
    feasibility_analysis = []
    
    for idx, row in best_scenarios.iterrows():
        changes = 0
        difficulty_score = 0
        
        # 비용/매출 변수 변화 체크
        for var in ['광고매출', '7일연속보상', '추가보상']:
            base_val = base_scenario.iloc[0][var]
            current_val = row[var]
            if current_val != base_val:
                changes += 1
                # 변화량에 따른 난이도 점수
                if var == '광고매출':
                    difficulty_score += abs(current_val - base_val) * 1  # 광고 수익 변경 난이도
                elif var == '7일연속보상':
                    difficulty_score += abs(current_val - base_val) / 10 * 2  # 보상 정책 변경 난이도
                elif var == '추가보상':
                    difficulty_score += abs(current_val - base_val) * 1.5  # 추가 보상 변경 난이도
        
        # 확률 변수 변화 체크 (백분율을 숫자로 변환)
        prob_vars = ['정답률', '광고시청률', '추가보상률', '재도전률']
        base_probs = [float(base_scenario.iloc[0][var].rstrip('%')) for var in prob_vars]
        current_probs = [float(row[var].rstrip('%')) for var in prob_vars]
        
        for i, var in enumerate(prob_vars):
            if current_probs[i] != base_probs[i]:
                changes += 1
                # 확률 변경 난이도 (변화량에 비례)
                difficulty_score += abs(current_probs[i] - base_probs[i]) / 10
        
        # 난이도 분류
        if difficulty_score <= 2:
            difficulty = "쉬움"
        elif difficulty_score <= 5:
            difficulty = "보통"
        else:
            difficulty = "어려움"
        
        feasibility_analysis.append({
            '시나리오_ID': row['시나리오_ID'],
            '변경_변수_수': changes,
            '난이도_점수': round(difficulty_score, 1),
            '달성_난이도': difficulty,
            '개선금액': row['개선금액']
        })
    
    feasibility_df = pd.DataFrame(feasibility_analysis)
    
    # 난이도별 분포
    difficulty_dist = feasibility_df['달성_난이도'].value_counts()
    print(f"달성 난이도 분포:")
    for difficulty, count in difficulty_dist.items():
        avg_improvement = feasibility_df[feasibility_df['달성_난이도'] == difficulty]['개선금액'].mean()
        print(f"  {difficulty}: {count}개 (평균 개선금액: {avg_improvement:,.0f}원)")
    
    # 가장 달성하기 쉬운 상위 3개 추천
    easy_scenarios = feasibility_df[feasibility_df['달성_난이도'] == '쉬움'].nlargest(3, '개선금액')
    if not easy_scenarios.empty:
        print(f"\n💡 달성하기 쉬운 추천 시나리오 (상위 3개):")
        for idx, scenario in easy_scenarios.iterrows():
            scenario_detail = best_scenarios[best_scenarios['시나리오_ID'] == scenario['시나리오_ID']].iloc[0]
            print(f"  {scenario['시나리오_ID']}: {scenario['개선금액']:,}원 개선 (변경 {scenario['변경_변수_수']}개 변수)")
            print(f"    설정: 광고={scenario_detail['광고매출']}, 연속보상={scenario_detail['7일연속보상']}, 추가보상={scenario_detail['추가보상']}")
            print(f"    확률: {scenario_detail['정답률']}, {scenario_detail['광고시청률']}, {scenario_detail['추가보상률']}, {scenario_detail['재도전률']}")
    
    return {
        'base_scenario': base_scenario.iloc[0],
        'best_scenarios': best_scenarios,
        'improved_count': len(improved_scenarios),
        'variable_analysis': variable_analysis,
        'feasibility_analysis': feasibility_df,
        'easy_recommendations': easy_scenarios if not easy_scenarios.empty else pd.DataFrame()
    }


def analyze_scenario_comparison(df, scenario_ids):
    """
    특정 시나리오들을 상세 비교 분석하는 함수
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Grid Search 결과 데이터프레임
    scenario_ids : list
        비교할 시나리오 ID 리스트
    """
    scenarios = df[df['시나리오_ID'].isin(scenario_ids)].copy()
    
    if scenarios.empty:
        print("❌ 해당 시나리오들을 찾을 수 없습니다.")
        return
    
    print(f"=== 시나리오 상세 비교 ({len(scenarios)}개) ===")
    
    # 주요 지표 비교
    comparison_columns = [
        '시나리오_ID', '광고매출', '7일연속보상', '추가보상',
        '정답률', '광고시청률', '추가보상률', '재도전률',
        '일일_광고시청수', '일일순손익액', '주간순손익액', '수익성여부'
    ]
    
    print(scenarios[comparison_columns].to_string(index=False))
    
    # 수익 구조 분석
    print(f"\n=== 수익 구조 분석 ===")
    
    for idx, row in scenarios.iterrows():
        print(f"\n{row['시나리오_ID']}:")
        if '광고매출액' in row:
            print(f"  매출: 광고 {row['광고매출액']:,}원")
        if all(col in row for col in ['일반보상액', '재도전보상액', '추가보상액', '7일연속보상액', '총보상액']):
            print(f"  비용: 일반 {row['일반보상액']:,}원 + 재도전 {row['재도전보상액']:,}원 + 추가 {row['추가보상액']:,}원 + 연속 {row['7일연속보상액']:,}원 = {row['총보상액']:,}원")
        if '주간순손익액' in row:
            print(f"  순이익: {row['주간순손익액']:,}원")


# 실행
if __name__ == "__main__":
    df = main()
    
    # DataFrame이 성공적으로 생성되었는지 확인
    if df is not None and len(df) > 0:
        print("\n" + "="*80)
        
        # 예시: 상위 10개 시나리오 추출
        try:
            best_analysis = extract_best_scenarios(df, n=10)
            
            if best_analysis:
                print("\n✅ 기본 시나리오 분석 완료")
            else:
                print("\n❌ 기본 시나리오 분석 실패")
        except Exception as e:
            print(f"\n❌ 기본 시나리오 분석 중 오류: {e}")
        
        # 예시: 특정 조건으로 시나리오 추출 (컬럼 존재 확인)
        print("\n" + "="*80)
        
        # 커스텀 기준 시나리오가 있는지 확인
        required_cols = ['광고매출', '7일연속보상', '추가보상', '정답률', '광고시청률', '추가보상률', '재도전률']
        if all(col in df.columns for col in required_cols):
            try:
                custom_base = {
                    '광고매출': 2,
                    '7일연속보상': 50,
                    '추가보상': 2,
                    '정답률': '60.0%',
                    '광고시청률': '60.0%',
                    '추가보상률': '80.0%',
                    '재도전률': '40.0%'
                }
                custom_analysis = extract_best_scenarios(df, base_scenario_params=custom_base, n=5, improvement_threshold=100000)
                
                if custom_analysis:
                    print("\n✅ 커스텀 시나리오 분석 완료")
                else:
                    print("\n❌ 커스텀 시나리오 분석 실패")
            except Exception as e:
                print(f"\n❌ 커스텀 시나리오 분석 중 오류: {e}")
        else:
            missing_cols = [col for col in required_cols if col not in df.columns]
            print(f"\n⚠️ 커스텀 분석을 위한 필수 컬럼 누락: {missing_cols}")
    else:
        print("\n❌ DataFrame 생성 실패")

=== 샘플 테스트 ===


TypeError: calculate_scenario_metrics() takes 5 positional arguments but 8 were given

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 414)